## Merging Multiple DataFrames to Combine JobsRetained & State Population

In this project, I am going to return only the 'State' and 'JobsRetained' features of the data as these are the only features from this data source that will need. For the purposes of this analysis, I am going to drop the samples that have missing values.

The source for the data: https://www.kaggle.com/susuwatari/ppp-loan-data-paycheck-protection-program.

In [1]:
import pandas as pd
import numpy as np

In [2]:
base_data = pd.read_csv('PPP_data_150k_plus.csv', engine='c', header=0,
                        usecols=['State', 'JobsRetained'], na_values=['XX', 'NaN'])

base_data = base_data.dropna()

base_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 620703 entries, 0 to 661201
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   State         620703 non-null  object 
 1   JobsRetained  620703 non-null  float64
dtypes: float64(1), object(1)
memory usage: 14.2+ MB


Next, I am going to ingest the 'State' and 'Code' features of the data below so that I can merge the dataset with population estimates as well as the dataset with the number of JobsRetained. Unfortunately, the two datasets list the States in different ways (abbreviated vs Spelled out).

In [3]:
code_to_state_name_df = pd.read_csv('csvData.csv', engine='c', usecols=[0,2])
code_to_state_name_df

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


After that, I am ingesting the State population information (only the state name and the Estimated Population features) that I retrieved from US Census site data (https://data.census.gov/cedsci/table?tid=PEPPOP2021.NST_EST2021_POP&hidePreview=false).

Before I display the output of the data, I rename the columns so they are more reader-friendly.

In [4]:
state_pop_df = pd.read_csv('US_Census_population-4-1-20.csv', usecols=[0,1], engine='c')
state_pop_df = state_pop_df.rename(columns={'Geographic Area Name (NAME)' : 'State', 
                                      'Estimates Base Population, April 1, 2020 (POP_BASE2020)' : 'Estimated Population'})
state_pop_df

,State,Estimated Population
0,United States,"331,449,281"
1,Northeast Region,"57,609,148"
2,Midwest Region,"68,985,454"
3,South Region,"126,266,107"
4,West Region,"78,588,572"
5,Oklahoma,"3,959,353"
6,Nebraska,"1,961,504"
7,Hawaii,"1,455,271"
8,South Dakota,"886,667"
9,Tennessee,"6,910,840"


In this section, I am going to aggregate (summate) the number of jobs saved and group them by State.

In [5]:
# do the aggregation first, then merge the dataframes
base_data = base_data.groupby(by='State').sum()

base_data['JobsRetained'] = base_data['JobsRetained'].astype(int)

base_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56 entries, AK to WY
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   JobsRetained  56 non-null     int64
dtypes: int64(1)
memory usage: 896.0+ bytes


Here, I am going to merge the two dataframes into one using an outer join based on the state name (spelled out).

After that, I am going to clean up the data. Actions taken include:
1) Reindex the features (reorder the features)

2) Create the 'Jobs Saved Per Population' feature and round it to 4 decimal places. The formula for creating this column is as follows:

    (JobsRetrained / Estimated Population)

3) Sort the data according to the number of job saved per population (in descending order)

In [6]:
full_data = base_data.merge(code_to_state_name_df, left_on='State', right_on='Code', how='outer').drop(columns=['Code']).drop(index=[]).merge(state_pop_df, left_on='State', right_on='State', how='outer').dropna().reset_index().drop(columns=['index'])

full_data = full_data.reindex(columns=['State', 'JobsRetained', 'Estimated Population'])

full_data['Estimated Population'] = full_data['Estimated Population'].str.replace(",","").astype(float)
full_data['Jobs Saved Per Population'] = round(full_data['JobsRetained']/full_data['Estimated Population'],4)

full_data.sort_values(by='Jobs Saved Per Population', ascending=False).reset_index().drop(columns=['index'])

,State,JobsRetained,Estimated Population,Jobs Saved Per Population
0,District of Columbia,123328.0,689545.0,0.1789
1,North Dakota,104376.0,779094.0,0.1340
2,Utah,422349.0,3271616.0,0.1291
3,Minnesota,711199.0,5706494.0,0.1246
4,South Dakota,101628.0,886667.0,0.1146
5,Kansas,318299.0,2937880.0,0.1083
6,Vermont,68659.0,643077.0,0.1068
7,Illinois,1360926.0,12812508.0,0.1062
8,Louisiana,492974.0,4657757.0,0.1058
9,Massachusetts,740747.0,7029917.0,0.1054


Finally, I am going the save the results of my work as a csv file as shown below.

In [7]:
full_data.to_csv('Best Jobs Retained Rates from PPP.csv')